In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Resize, Compose
from torchvision.datasets import Cityscapes
from torch.utils.data import random_split

In [2]:

transform = Compose([
    Resize((256, 256)),
    ToTensor()
])


target_transform = Compose([
    Resize((256, 256)),
    ToTensor()
])

dataset = Cityscapes(root="E:/CityScapes", split='train', mode='fine', target_type='semantic', transform=transform, target_transform=target_transform)


train_dataset, val_dataset = random_split(dataset, [0.8,0.2])

In [3]:
train_dataset[0][1].size()

torch.Size([1, 256, 256])

In [4]:
batch_size = 2

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [5]:

from model import Model

model = Model().cuda()
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [9]:
def train(dataloader, model, loss_fn, optimizer):
    """
    Train a model for 1 epoch.

    Params:
    - dataloader:   dataset to train on.
    - model:        the model object to be trained.
    - loss_fn:      the loss function.
    - optimizer:    the desired optimization.
    """
    size = len(dataloader.dataset)
    model.train() #Set the model to train mode
    for batch, (IMG,SEGM) in enumerate(dataloader):
        IMG = IMG.to('cuda')
        SEGM = SEGM.to('cuda')

        
        #predict
        pred = model(IMG)
        #Loss
        loss = loss_fn(pred, SEGM)

        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #print loss during training
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(IMG)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [7]:
def test(dataloader, model, loss_fn):
    """
    Test a model.

    Params:
    - dataloader:   dataset to test on.
    - model:        the model object to be tested.
    - loss_fn:      the loss function.
    """
    num_batches = len(dataloader)
    model.eval() #model in eval mode
    test_loss = 0
    with torch.no_grad():
        for _, (X,_) in enumerate(dataloader):
            X = X.to('cuda')

            pred = model(X)
            test_loss += loss_fn(pred, X).item()
            
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")


In [10]:
epochs = 1 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(val_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
torch.Size([2, 34, 256, 256])


c:\Users\Sven\anaconda3\envs\Q2-5LSH0\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2, 1, 256, 256])) that is different to the input size (torch.Size([2, 34, 256, 256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 0.166176  [    2/ 2380]
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size([2, 34, 256, 256])
torch.Size

KeyboardInterrupt: 